In [4]:
import pygame
import sys
from math import *
import random
import numpy as np

pygame.init()
width = 500
height = 660
main_surface = pygame.display.set_mode((width, height))
clock = pygame.time.Clock()
#大分子圓球 橘色
orange = (255,165,0)
#小分子圓球 紅色
red = (255,0,0)
balls = []
g=0
vx0 = np.random.normal(1,0.3)*0.01
vy0 = np.random.normal(1,0.3)*0.01
ratio=0.9
# Ball Class
class Ball:
    def __init__(self, x, y, vx, vy, color, hard, radius, mass):
        self.x = x
        self.y = y
        self.color = color
        self.vx = vx
        self.vy = vy
        self.hard = hard
        self.radius = radius
        self.mass = mass

    # Draws Balls on Display Window
    def draw(self, x, y):
        pygame.draw.circle(main_surface, self.color, [int(x - self.radius), int(y - self.radius)], self.radius)

    # Moves the Ball around the Screen
    def move(self):
        
        '''if self.hard == 1:
            self.vx = 0
            self.vy = 0'''
        self.x = self.x + self.vx
        self.y = self.y + self.vy

        if (self.x + self.radius >= width):
            self.x = width - self.radius
            self.vx = -self.vx
        if (self.radius >= self.x):
            self.x = self.radius
            self.vx = -self.vx
        if (self.y + self.radius >= height):
            self.y = height - self.radius
            self.hard = 1
#             self.angle = 360 - self.angle
            self.vy = -self.vy
        if (self.radius >= self.y):
            self.y = self.radius
            self.vy=-self.vy

def collision(ball1, ball2):
    r12 = [ball2.x-ball1.x, ball2.y-ball1.y]
    v1 = [ball1.vx, ball1.vy]
    v2 = [ball2.vx, ball2.vy]
    dist = ((ball1.x - ball2.x)**2 + (ball1.y - ball2.y)**2)**0.5
    if dist <= ball1.radius + ball2.radius:
        
        if  r12[0]*v2[0] + r12[1]*v2[1] > 0 and r12[0]*v1[0] + r12[1]*v1[1] < 0 :
        
            return False
        elif (r12[0]*v2[0] + r12[1]*v2[1])*(r12[0]*v1[0] + r12[1]*v1[1]) > 0 :
            if r12[0]*v2[0] + r12[1]*v2[1] > 0 and (r12[0]*v1[0] + r12[1]*v1[1])-(r12[0]*v2[0] + r12[1]*v2[1])<0:
                return False
            elif r12[0]*v2[0] + r12[1]*v2[1] < 0 and (r12[0]*v1[0] + r12[1]*v1[1])-(r12[0]*v2[0] + r12[1]*v2[1])>0:
                return False
        elif r12[0]*v1[0] + r12[1]*v1[1] < 0:
            return False
        else:
            return True
    
    else:
        return False
    
# Checks Collision Between Balls
def checkCollision():
    for i in range(len(balls)):
        for j in range(len(balls) - 1, i, -1):
            if collision(balls[i], balls[j]):
                vx1 = balls[i].vx
                vx2 = balls[j].vx
                vy1 = balls[i].vy
                vy2 = balls[j].vy
                v1 = (balls[i].vx**2+balls[i].vy**2)**0.5
                v2 = (balls[j].vx**2+balls[j].vy**2)**0.5
                if vx1 or vx2 or vy1 or vy2 == 0:
                    ang1 = pi/2
                    ang2 = pi/2
                    
                else:
                    ang1 = acos(vx1/v1)
                    ang2 = acos(vx2/v2)
                m1 = balls[i].mass
                m2 = balls[j].mass
                
                if balls[i].x == balls[j].x:
                    phi = pi/2
                else:
                    phi = atan((balls[j].y-balls[i].y)/(balls[j].x-balls[i].x))
                phi2 = pi-phi
                vxr1 = v1*cos(ang1-phi)
                vyr1 = v1*sin(ang1-phi)
                vxr2 = v2*cos(ang2-phi)
                vyr2 = v2*sin(ang2-phi)
                vfxr1 = ((m1-m2)*vxr1+(2*m2)*vxr2)/(m1+m2)
                vfxr2 = ((2*m1)*vxr1+(m2-m1)*vxr2)/(m1+m2)
                vfyr1 = vyr1
                vfyr2 = vyr2
                vfx1 = cos(phi)*vfxr1 + cos(phi+pi/2)*vfyr1
                vfy1 = sin(phi)*vfxr1 + sin(phi+pi/2)*vfyr1
                vfx2 = cos(phi)*vfxr2 + cos(phi+pi/2)*vfyr2
                vfy2 = sin(phi)*vfxr2 + sin(phi+pi/2)*vfyr2
                balls[i].vx = vfx1
                balls[i].vy = vfy1
                balls[j].vx = vfx2
                balls[j].vy = vfy2
               
                    
                #balls[i].vx = (v1*cos(theata1-phi)*(m1-m2) + 2*m2*v2*cos(theata2-phi))*cos(phi)/(m1 + m2) + v1*sin(theata1-phi)*sin(phi)
                #balls[i].vy = (v1*cos(theata1-phi)*(m1-m2) + 2*m2*v2*cos(theata2-phi))*sin(phi)/(m1 + m2) + v1*sin(theata1-phi)*cos(phi)
                #balls[j].vx = (v2*cos(theata2-phi2)*(m2-m1) + 2*m1*v1*cos(theata1-phi2))*cos(phi2)/(m1 + m2) + v2*sin(theata2-phi2)*sin(phi2)
                #balls[j].vy = (v2*cos(theata2-phi2)*(m1-m2) + 2*m1*v1*cos(theata1-phi2))*sin(phi2)/(m1 + m2) + v2*sin(theata2-phi2)*cos(phi2)
                
                #balls[j].vx = vx1*m1/m2 + vx2 -(m1/m2)*((v1*cos(theata1-phi)*(m1-m2) + 2*m2*v2*cos(theata2-phi))*cos(phi)/(m1 + m2) + v1*sin(theata1-phi)*sin(phi))
                #balls[j].vy = vy1*m1/m2 + vy2 -(m1/m2)*((v1*cos(theata1-phi)*(m1-m2) + 2*m2*v2*cos(theata2-phi))*sin(phi)/(m1 + m2) + v1*sin(theata1-phi)*cos(phi))
def reset():
    global balls
    # (x, y, vx, vy, color, hard, radius, mass)
    # (x, y, speed, color, angle, hard, radius, mass)
    s = 100
    for i in range(2):
        b = Ball(s + i*100, 31, 0+10*np.random.normal(2,0.3)*i, 0*np.random.normal(2,0.3), orange, 0, 10+50*i, 5)
        balls.append(b)

def close():
    pygame.quit()
    sys.exit()
    
def main():
    reset()
    while True:
        event = pygame.event.poll()
        if event.type == pygame.QUIT:
            close()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                close()

            if event.key == pygame.K_r:
                main()


        
        main_surface.fill((0,0,0))
        
        for i in range(len(balls)):
            balls[i].move()
        '''for i in range(len(balls)):
            balls[i].vy=balls[i].vy+g'''

        checkCollision()
        for i in range(len(balls)):
            balls[i].vy=balls[i].vy+g

        for i in range(len(balls)):
            balls[i].draw(balls[i].x, balls[i].y)
            
        for i in range(len(balls)):
            balls[i].move()
        pygame.display.update()
        clock.tick(10)
        
main()


SystemExit: 